In [1]:
%load_ext autoreload

%autoreload 2

from RNN2DFA.LSTM import LSTMNetwork
# from GRU import GRUNetwork
from RNN2DFA.RNNClassifier import RNNClassifier
from RNN2DFA.Training_Functions import mixed_curriculum_train
import Tomita_Grammars 
from lstar_extraction.Training_Functions import make_test_set,make_train_set_for_target
from RNNexplainer import Explainer
import pandas as pd
import LTL2DFA as ltlf2dfa





In [2]:
from specific_examples import Alternating_Bit_Protocol
abp=Alternating_Bit_Protocol()
generator_dfa=abp.dfa
target_formula=abp.target_formula
alphabet=generator_dfa.alphabet



In [3]:



# make training set on grammars
# target = Tomita_Grammars.tomita_email
# alphabet = "pqradmn"
# generator_dfa.classify_word=target
# target_formula="mail string match"


# use a dfa to generate training set
target_formula="F(a & X(b))"
alphabet="abc"
fout=open("output/log.txt", "a")
fout.write(".........................................................................\n")
fout.write("Target: "+ target_formula)
fout.write("\n")
fout.close()
import LTL2DFA as ltlf2dfa
generator_dfa=ltlf2dfa.translate_ltl2dfa(alphabet=[character for character in alphabet],formula=target_formula)
print(generator_dfa)





# print(train_set)
# print("\n\nPositive examples:")
# for key in train_set:
#     if(train_set[key]):
#         print(key)
# print(train_set)


DFA:->
 - alphabet: ['a', 'b', 'c']
 - Q: [1, 2, 3, 4]
 - q0: 1
 - F: [4]
 - delta: {1: {'000': 2, '001': 2, '010': 2, '011': 2, '100': 3, '101': 3, '110': 3, '111': 3}, 2: {'000': 2, '001': 2, '010': 2, '011': 2, '100': 3, '101': 3, '110': 3, '111': 3}, 3: {'000': 2, '001': 2, '010': 4, '011': 4, '100': 3, '101': 3, '110': 4, '111': 4}, 4: {'000': 4, '001': 4, '010': 4, '011': 4, '100': 4, '101': 4, '110': 4, '111': 4}}
 - is_singleton_graph: False


In [4]:
# make training set
train_set = make_train_set_for_target(generator_dfa.classify_word,alphabet,max_train_samples_per_length=100,search_size_per_length=300)


made train set of size: 2213 , of which positive examples: 1254


In [5]:


# define rnn
rnn = RNNClassifier(alphabet,num_layers=1,hidden_dim=10,RNNClass = LSTMNetwork)


# train the model
mixed_curriculum_train(rnn,train_set,stop_threshold = 0.0005)
rnn.renew()  
dfa_from_rnn=rnn 
# statistics

def percent(num,digits=2):
    tens = pow(10,digits)
    return int(100*num*tens)/tens

test_set = train_set 
print("testing on train set, i.e. test set is train set")
# we're printing stats on the train set for now, but you can define other test sets by using
# make_train_set_for_target

n = len(test_set)
print("test set size:", n)
pos = len([w for w in test_set if generator_dfa.classify_word(w)])
print("of which positive:",pos,"("+str(percent(pos/n))+")")
rnn_target = len([w for w in test_set if dfa_from_rnn.classify_word(w)==generator_dfa.classify_word(w)])
print("rnn score against target on test set:                             ",rnn_target,"("+str(percent(rnn_target/n))+")")

# dfa_from_rnn=generator_dfa

The dy.parameter(...) call is now DEPRECATED.
        There is no longer need to explicitly add parameters to the computation graph.
        Any used parameter will be added automatically.
current average loss is:  0.0021303590091186738
classification loss on last batch was: 0.000484731609962767
testing on train set, i.e. test set is train set
test set size: 2213
of which positive: 1254 (56.66)
rnn score against target on test set:                              2213 (100.0)


In [8]:


# use a query LTL formula
query_formula="F(a)"
query_dfa=ltlf2dfa.translate_ltl2dfa(alphabet=[character for character in alphabet],formula=query_formula)

"""  
Create initial samples
"""

test_set=[]
if(query_dfa is None):
    query_formula=None
    test_set=make_test_set(alphabet)
    raise SystemError


from RNNexplainer import Traces
traces=Traces(rnn, alphabet)
traces.label_from_network(test_set)
traces.write_in_file()




from PACTeacher.pac_teacher import PACTeacher as Teacher 
explainer=Explainer(alphabet=[character for character in alphabet])
teacher = Teacher(dfa_from_rnn,epsilon=.05, delta=.05, max_trace_length=20, max_formula_depth=10, query_dfa=query_dfa)



import time
start_time=time.time()
from multiprocessing import Process, Queue
explainer, flag= teacher.teach(explainer, traces, timeout = 15)
end_time=time.time()


print("\n\nepsilon=", teacher.epsilon, "delta=", teacher.delta, "max_trace_length=", teacher.max_trace_length)
print("query:", query_formula)
print("final ltl: ", explainer.ltl)

fout=open("output/log.txt", "a")
fout.write("\n\nquery: "+query_formula)
fout.write("\nfinal LTL: "+ explainer.ltl)
if(not flag):
    fout.write(" [incomplete]")
    print("incomplete formula")
fout.write("\n\n")

print("\nTime taken:", end_time-start_time)
fout.close()


test_set = train_set 
fout=open("output/log.txt", "a")
# fout.write("rnn score against target on test set:                             "+str(rnn_target)+"("+str(percent(rnn_target/n))+")")
# fout.write("\n")

performance_ltl = len([w for w in test_set if dfa_from_rnn.classify_word(w)==explainer.dfa.classify_word(w)])
print("extracted LTL score against rnn on test set:                      ",performance_ltl,"("+str(percent(performance_ltl/n))+")")
performance_ltl_with_target = len([w for w in test_set if explainer.dfa.classify_word(w)==generator_dfa.classify_word(w)])
print("extracted LTL score against target on rnn's test set:             ",performance_ltl_with_target,"("+str(percent(performance_ltl_with_target/n))+")")

performance_ltl = len([w for w in test_set if (dfa_from_rnn.classify_word(w)and query_dfa.classify_word(w)) ==explainer.dfa.classify_word(w)])
print("extracted LTL score against rnn on test set (with query):         ",performance_ltl,"("+str(percent(performance_ltl/n))+")")
# fout.write("extracted LTL score against rnn on test set (with query):         "+str(performance_ltl)+"("+str(percent(performance_ltl/n))+")\n")
performance_ltl_with_target = len([w for w in test_set if explainer.dfa.classify_word(w)== (generator_dfa.classify_word(w) and query_dfa.classify_word(w))])
print("extracted LTL score against target on rnn's test set (with query):",performance_ltl_with_target,"("+str(percent(performance_ltl_with_target/n))+")")

# fout.write("extracted LTL score against target on rnn's test set (with query):"+str(performance_ltl_with_target)+"("+str(percent(performance_ltl_with_target/n))+")\n")
fout.close()


# report in a pandas file
result = pd.DataFrame(columns=['target', 
                                'query', 
                                'explanation', 
                                'status', 
                                'rnn score', 
                                'explanation score', 
                                'explanation score on ground truth',
                                'extraction time'
                                ])

result = result.append(
    {
        'target':target_formula,
        'query':query_formula,
        'explanation':explainer.ltl,
        'status':flag,
        'rnn score':percent(rnn_target/n),
        'explanation score':percent(performance_ltl/n),
        'explanation score on ground truth':percent(performance_ltl_with_target/n),
        'extraction time': end_time-start_time
    }, ignore_index=True
)
print(result.to_string(index=False))
result.to_csv('output/result.csv', header=False, index=False, mode='a')




positive traces---> 
[]


negative traces---> 
[]



learned LTL formula: true
new counterexample:   should be rejected by implementation


positive traces---> 
[]


negative traces---> 
['']



0  iteration complete



learned LTL formula: false
new counterexample: ab  should be accepted by implementation


positive traces---> 
['ab']


negative traces---> 
['']



1  iteration complete



start formula depth: 1
learned LTL formula: a
new counterexample: a  should be rejected by implementation


positive traces---> 
['ab']


negative traces---> 
['', 'a']



2  iteration complete



start formula depth: 1
learned LTL formula: (X b)
new counterexample: aab  should be accepted by implementation


positive traces---> 
['ab', 'aab']


negative traces---> 
['', 'a']



3  iteration complete



start formula depth: 2
learned LTL formula: (F b)
new counterexample: b  should be rejected by implementation


positive traces---> 
['ab', 'aab']


negative traces---> 
['', 'a', 'b']



4  iterat

In [7]:
# read result
# df=pd.read_csv("output/result.csv",header=None)
# df.columns=['target', 
#             'query', 
#             'explanation', 
#             'status', 
#             'rnn score', 
#             'explanation score', 
#             'explanation score on ground truth',
#             'extraction time'
#             ]
# print(df.to_string(index=False))

In [21]:
import signal
from contextlib import contextmanager


@contextmanager
def timeout(time):
    # Register a function to raise a TimeoutError on the signal.
    signal.signal(signal.SIGALRM, raise_timeout)
    # Schedule the signal to be sent after ``time``.
    signal.alarm(time)

    try:
        yield
    except TimeoutError:
        pass
    finally:
        # Unregister the signal so it won't be triggered
        # if the timeout is not reached.
        signal.signal(signal.SIGALRM, signal.SIG_IGN)


def raise_timeout(signum, frame):
    raise TimeoutError

import time

def another_func():
    print('entering block')
def my_func():
    # Add a timeout block.
    with timeout(10):
        i=0
        while(i<2):
            another_func()
            i+=1
            time.sleep(3)
            print('This should never get printed because the line before timed out')
    print("program executed")
my_func()

entering block
This should never get printed because the line before timed out
entering block
This should never get printed because the line before timed out
program executed
